In [11]:
import ConfigReader as cr
import DatafileIO as dio
import pandas as pd

ini_path = "tests/config/load_pythagorean_triples_all.ini"

cp = cr.read_config(ini_path)

In [12]:
functions = cr.parse_functions(cp)
file1 = cp["control"]["file1"]
col1 = cp["control"]["column1"]
file2 = cp["control"]["file2"]
col2 = cp["control"]["column2"]

print(functions, file1, col1, file2, col2, sep = "\n")

nfiles = int(file1 != "") + int(file2 != "")
ncols = int(col1 != "") + int(col2 != "")

print(nfiles, ncols, sep = "\n")

['DatafileIO.load_files_from_ini']
tests/data/pythagorean_triples.csv



1
0


In [13]:
a = dio.load_files_from_ini(ini_path)
b = pd.read_csv(ini_path)

In [18]:
a.equals(b)
a
b

,[info]
0,description = Load pythagorean_triples.csv tes...
1,[control]
2,functions = DatafileIO.load_files_from_ini
3,file1 = tests/data/pythagorean_triples.csv
4,column1 =
5,file2 =
6,column2 =


In [33]:
config_path = "tests/config/"

# Return full path to test ini and test data file
def _get_paths(ini):
    ini_path = config_path + ini
    cp = cr.read_config(ini_path)
    data_path = "src/" + str(cp["control"]["file1"])
    return ini_path, data_path


In [34]:
ini_path, data_path = _get_paths("load_pythagorean_triples_all.ini")
print(ini_path, data_path, sep = "\n")

tests/config/load_pythagorean_triples_all.ini
src/tests/data/pythagorean_triples.csv


In [30]:
data_path

'src/tests/data/pythagorean_triples.csv'